In [47]:
import numpy as np
import pandas as pd


In [48]:
df_test = pd.read_csv('/content/sttest.csv')
df_train = pd.read_csv('/content/sttrain.csv')

In [49]:
result = df_test[["PassengerId"]]

In [50]:
#Passenger ID , Name
df_train = df_train.drop(['PassengerId', 'Name'], axis=1)
df_test = df_test.drop(['PassengerId', 'Name'], axis=1)

In [51]:
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

In [52]:
X_train = df_train.drop('Transported', axis=1)
y_train = df_train['Transported']

In [53]:
df_train.isnull().sum()

,0
HomePlanet,201
CryoSleep,217
Cabin,199
Destination,182
Age,179
VIP,203
RoomService,181
FoodCourt,183
ShoppingMall,208
Spa,183


In [54]:
X_train.sample(2)

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
8074,Earth,False,G/1392/S,TRAPPIST-1e,38.0,False,1739.0,0.0,0.0,83.0,0.0
5992,Mars,True,E/423/S,TRAPPIST-1e,42.0,False,0.0,0.0,0.0,0.0,0.0


In [55]:
count = 0
Categorical_index_list = []
Numerical_index_list = []
for i in X_train.columns:
    if X_train[i].isnull().sum() > 0:
        if X_train[i].dtype == 'object':
            Categorical_index_list.append(count)
        else:
            Numerical_index_list.append(count)
        count += 1
print(Categorical_index_list)
print(Numerical_index_list)

[0, 1, 2, 3, 5]
[4, 6, 7, 8, 9, 10]


In [56]:
impute = ColumnTransformer([
    ('Impute_cat', SimpleImputer(strategy = 'most_frequent'), Categorical_index_list),
    ('Impute_num', SimpleImputer(strategy = 'mean'), Numerical_index_list)
    ], remainder='passthrough')


In [11]:
# impute.fit(X_train)
# X_train = pd.DataFrame(impute.transform(X_train))
# X_train.columns = df_train.drop('Transported', axis =1).columns
# X_train.isnull().sum()

In [57]:
list(X_train.columns)

['HomePlanet',
 'CryoSleep',
 'Cabin',
 'Destination',
 'Age',
 'VIP',
 'RoomService',
 'FoodCourt',
 'ShoppingMall',
 'Spa',
 'VRDeck']

In [58]:
def combine_columns(df):
    df = pd.DataFrame(df)
    columns = ['HomePlanet','CryoSleep','Cabin','Destination','Age','VIP',
               'RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']
    df.columns = columns
    df['Spent'] = df['FoodCourt'] + df['ShoppingMall'] + df['Spa'] + df['VRDeck'] + df['RoomService']
    df.drop(['FoodCourt','ShoppingMall','Spa','VRDeck','RoomService'], axis=1, inplace=True)
    return df

In [59]:
def split_cabin(df):
    df = pd.DataFrame(df)
    df[['Deck','Num','Side']] = df['Cabin'].str.split('/', expand=True)
    df.drop('Cabin', axis=1, inplace=True)
    return df

In [60]:
def convert(X_test):
  X_test = pd.DataFrame(X_test)
  for col in X_test.select_dtypes(include=['object']):
    try:
        X_test[col] = X_test[col].astype(float)  # Try converting to float
    except ValueError:
        X_test[col] = X_test[col].astype('category')
  return X_test

In [61]:
combine_transformer = FunctionTransformer(combine_columns)
cabin_splitter = FunctionTransformer(split_cabin)
convert_transformer = FunctionTransformer(convert)

In [62]:
OHE = ColumnTransformer([
    ('OHE_cat', OneHotEncoder(sparse_output = False, handle_unknown= 'ignore'), [0,2,6,8])
], remainder = 'passthrough')

In [63]:
Scaling = ColumnTransformer([
    ('scale',StandardScaler(),slice(0,20))
], remainder = 'passthrough')

In [64]:
import xgboost as xgb
model = xgb.XGBClassifier(objective='binary:logistic')

In [65]:
pipeline = Pipeline([
    ('imputation', impute),
    ('combine_columns', combine_transformer),
    ('cabin_splitter', cabin_splitter),
    ('One Hot', OHE),
    ('Scaling', Scaling),
    ('convert', convert_transformer),
    ('model', model)
])

Prev

In [66]:
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(df_test)

In [67]:
y_pred

array([1, 1, 1, ..., 1, 0, 0])

In [68]:
result['Transported'] = y_pred

In [69]:
result['Transported'] = result['Transported'].astype(bool)

In [70]:
result.to_csv('submission.csv', index=False)